# Implementeren van een Dieren Expert Systeem

Een voorbeeld uit [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

In dit voorbeeld implementeren we een eenvoudig kennisgebaseerd systeem om een dier te bepalen op basis van enkele fysieke kenmerken. Het systeem kan worden weergegeven door de volgende AND-OR-boom (dit is een deel van de volledige boom, we kunnen eenvoudig meer regels toevoegen):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.nl.png)


## Onze eigen expertensysteem-shell met achterwaartse inferentie

Laten we proberen een eenvoudige taal te definiëren voor kennisrepresentatie op basis van productieregels. We zullen Python-klassen gebruiken als trefwoorden om regels te definiëren. Er zijn in wezen drie soorten klassen:
* `Ask` vertegenwoordigt een vraag die aan de gebruiker moet worden gesteld. Het bevat de set mogelijke antwoorden.
* `If` vertegenwoordigt een regel en is slechts syntactische suiker om de inhoud van de regel op te slaan.
* `AND`/`OR` zijn klassen om AND/OR-takken van de boom te vertegenwoordigen. Ze slaan gewoon de lijst van argumenten op. Om de code te vereenvoudigen, is alle functionaliteit gedefinieerd in de bovenliggende klasse `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

In ons systeem zou het werkgeheugen de lijst van **feiten** bevatten als **attribuut-waardeparen**. De kennisbasis kan worden gedefinieerd als één grote woordenlijst die acties (nieuwe feiten die in het werkgeheugen moeten worden ingevoegd) koppelt aan voorwaarden, uitgedrukt als EN-OF-expressies. Ook kunnen sommige feiten worden `Gevraagd`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Om de achterwaartse inferentie uit te voeren, definiëren we de klasse `Knowledgebase`. Deze zal bevatten:
* Werkgeheugen (`memory`) - een woordenboek dat attributen aan waarden koppelt
* Kennisbankregels (`rules`) in het hierboven gedefinieerde formaat

Twee belangrijkste methoden zijn:
* `get` om de waarde van een attribuut te verkrijgen, waarbij indien nodig inferentie wordt uitgevoerd. Bijvoorbeeld, `get('color')` haalt de waarde van een kleur-slot op (het zal indien nodig vragen stellen en de waarde opslaan voor later gebruik in het werkgeheugen). Als we `get('color:blue')` vragen, zal het om een kleur vragen en vervolgens een `y`/`n`-waarde retourneren afhankelijk van de kleur.
* `eval` voert de daadwerkelijke inferentie uit, d.w.z. doorloopt de AND/OR-boom, evalueert subdoelen, enzovoort.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Laten we nu onze dierenkennisbank definiëren en de consultatie uitvoeren. Merk op dat deze oproep u vragen zal stellen. U kunt antwoorden door `y`/`n` te typen voor ja-nee vragen, of door een nummer (0..N) op te geven voor vragen met langere meerkeuzeantwoorden.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## PyKnow gebruiken voor Forward Inference

In het volgende voorbeeld gaan we proberen forward inference te implementeren met behulp van een van de bibliotheken voor kennisrepresentatie, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** is een bibliotheek voor het creëren van forward inference-systemen in Python, die is ontworpen om vergelijkbaar te zijn met het klassieke oude systeem [CLIPS](http://www.clipsrules.net/index.html).

We hadden forward chaining ook zelf kunnen implementeren zonder al te veel problemen, maar eenvoudige implementaties zijn meestal niet erg efficiënt. Voor effectievere regelmatching wordt een speciaal algoritme, [Rete](https://en.wikipedia.org/wiki/Rete_algorithm), gebruikt.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

We zullen ons systeem definiëren als een klasse die `KnowledgeEngine` subclass. Elke regel wordt gedefinieerd door een aparte functie met de annotatie `@Rule`, die specificeert wanneer de regel moet worden uitgevoerd. Binnen de regel kunnen we nieuwe feiten toevoegen met de functie `declare`, en het toevoegen van die feiten zal ertoe leiden dat meer regels worden aangeroepen door de forward inference engine.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Zodra we een kennisbasis hebben gedefinieerd, vullen we ons werkgeheugen met enkele initiële feiten en roepen vervolgens de methode `run()` aan om de inferentie uit te voeren. Je kunt als resultaat zien dat nieuwe afgeleide feiten aan het werkgeheugen worden toegevoegd, inclusief het uiteindelijke feit over het dier (als we alle initiële feiten correct hebben ingesteld).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
